In [1]:
# !pip install ipywidgets
import numpy as np
from scipy.io import wavfile as wave
from scipy.fft import dct
from scipy import signal
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, Layout
from IPython.display import display, HTML

In [2]:
# Get data from .wav file
def get_time_series(path_to_file):
    info = wave.read(path_to_file)
    amp = info[1][:, 0]
    amp = amp / amp.max()
    time = np.linspace(0., len(amp) / info[0], len(amp))
    return [time, amp]

# Get envelope
def get_envelope(time_series):
    peaks = signal.find_peaks(time_series[1], threshold = 0.00005, distance = 1000)[0]
    t = [time_series[0][x] for x in peaks]
    peaks = [time_series[1][x] for x in peaks]
    return [t, peaks]

In [3]:
# Apply Fourier transform
def fourier_transform(time_series):
    amp = time_series[1]
    window = signal.get_window('hann', len(amp))
    amp = amp * window
    fft_amp = abs(dct(amp) / len(amp))
    freq = np.arange(len(amp)) / time_series[0][-1] / 2
    return [freq, fft_amp]

# Get amplitudes of f0, f1, f2, etc. from frequency domain
def get_timbre(freq_series):
    indexes = signal.find_peaks(freq_series[1], threshold = 0.0005, distance = 1000)[0]
    freq = freq_series[0][indexes[:7]]
    amp = freq_series[1][indexes[:7]]
    return [freq, amp]

In [4]:
# Linear function (y = a * x + b)
def linear_f(x, a, b):
    return a * x + b

# Fit linear function to data
def linear_fit(time_series, first, last):
    region = [time_series[0][first:last], time_series[1][first:last]]
    envelope = get_envelope(region)
    sigma = np.ones(len(envelope[0]))
    sigma[[0, -1]] = 0.2
    coefficients, _ = curve_fit(linear_f, envelope[0], envelope[1], sigma=sigma)
    curve = []
    for i in range(len(region[0])):
        curve.append(linear_f(region[0][i], coefficients[0], coefficients[1]))
    return ["y = %.3fx + %.3f" % (coefficients[0], coefficients[1]), curve, first, last]

In [5]:
# Exponential function (y = a * b ^ x)
def exp_f(x, a, b):
    return np.sign(b) * a * (np.abs(b) ** x)

# Fit exponential function to data
def exponential_fit(time_series, first, last):
    region = [time_series[0][first:last], time_series[1][first:last]]
    envelope = get_envelope(region)
    sigma = np.ones(len(envelope[0]))
    sigma[[0, -1]] = 0.2
    coefficients, _ = curve_fit(exp_f, envelope[0], envelope[1], sigma=sigma)
    curve = []
    for i in range(len(region[0])):
        curve.append(exp_f(region[0][i], coefficients[0], coefficients[1]))
    return ["y = %.3f &#8729; %.3f<sup>x<sup>" % (coefficients[0], coefficients[1]), curve, first, last]

In [6]:
# Map strings to function references 
function_type_map = {
    'Linear' : linear_fit,
    'Exponential' : exponential_fit,
}

# Fit a model
def function_fit(funct, time_series, first, last):
    return funct(time_series, first, last)


def create_ADSR(time_series, A, D, S, type_A, type_D, type_R):
    result = {
        'A' : [],
        'D' : [],
        'S' : [],
        'R' : [],
    }
    if not (0 <= A + D + S < 1):
        print('invalid ADS input')
        return
    
    range_A = int(A * len(time_series[0]))
    range_D = int(D * len(time_series[0]))
    range_S = int(S * len(time_series[0]))
    
    if A != 0:
        A_fit = function_fit(function_type_map[type_A], time_series, 0, range_A)
        result['A'] = A_fit
    if D != 0: 
        D_fit = function_fit(function_type_map[type_D], time_series, range_A, range_A + range_D)
        result['D'] = D_fit
    if S != 0:
        S_fit =  ["y = %.3f" %(time_series[1][range_A + range_D - 1]), [time_series[1][range_A + range_D - 1]] * range_S, range_A + range_D, range_A + range_D + range_S]
        result['S'] = S_fit
    R_fit = function_fit(function_type_map[type_R], time_series, range_A + range_D + range_S, len(time_series[0]))
    result['R'] = R_fit

    return result

In [7]:
path_to_file = 'C3-Guitar.wav'
time_series = get_time_series(path_to_file)
freq_series = fourier_transform(time_series)
envelope = get_envelope(time_series)
timbre = get_timbre(freq_series)

def plot_everything(A, D, S, type_A, type_D, type_R):
    ADSR_envelope = create_ADSR(time_series, A, D, S, type_A, type_D, type_R)
    ADSR_fit = []
    for key in ADSR_envelope:
        if ADSR_envelope[key] != []:
            ADSR_fit += ADSR_envelope[key][1]
            equation = '<div style="position: relative; left: 50px; margin: 0 0 10px 0">' + key + ':    ' + ADSR_envelope[key][0] + '</div>'
            display(HTML(equation))
            
    # Create figure
    fig = plt.figure(figsize=(12,9))

    # Time Domain
    time_series_plot = fig.add_subplot(211)

    # Formatting
    time_series_plot.title.set_text('Time Domain')
    time_series_plot.set_xlabel('Time (s)')
    time_series_plot.set_ylabel('Amplitude')

    # Plotting
    time_series_plot.plot(time_series[0], time_series[1], label="Time series", color="cyan")
    time_series_plot.plot(envelope[0], envelope[1], label="Envelope")
    time_series_plot.plot(time_series[0], ADSR_fit, label="ADSR", color="red")
    time_series_plot.legend()

    # Frequency Domain
    freq_series_plot = fig.add_subplot(212)

    # Formatting
    freq_series_plot.title.set_text('Frequency Domain')
    freq_series_plot.set_xlabel('Frequency (Hz)')
    freq_series_plot.set_ylabel('Amplitude')
    freq_series_plot.set_ylim(0, freq_series[1].max() * 1.5)
    ylim = freq_series_plot.get_ylim()
    freq_series_plot.margins(0, 0.02)

    # Plotting
    freq_series_plot.scatter(timbre[0], timbre[1], color = "red")
    for x, y in zip(timbre[0], timbre[1]):
        freq_series_plot.annotate("(%.2f, %.4f)" % (x, y), (x, y + abs(ylim[1]-ylim[0]) / 20), ha='center', va='center')
    freq_series_plot.plot(freq_series[0][0:10000], freq_series[1][:10000], label="Frequency series")
    freq_series_plot.legend()

    # Layout
    fig.tight_layout()

interact(
    plot_everything, 
    A=widgets.FloatSlider(value=0, min=0, max=1, step=0.005, readout_format='.3f', layout=Layout(width='800px')),
    D=widgets.FloatSlider(value=0, min=0, max=1, step=0.005, readout_format='.3f', layout=Layout(width='800px')),
    S=widgets.FloatSlider(value=0, min=0, max=1, step=0.005, readout_format='.3f', layout=Layout(width='800px')),
    type_A=widgets.Dropdown(options=['Linear', 'Exponential'], value='Linear', description='Type of A', layout=Layout(margin='20px 0px 0px 0px', left='2px')),
    type_D=widgets.Dropdown(options=['Linear', 'Exponential'], value='Linear', description='Type of D'),
    type_R=widgets.Dropdown(options=['Linear', 'Exponential'], value='Linear', description='Type of R', layout=Layout(margin='0px 0px 20px 0px', left='2px'))
)
    


<ipython-input-2-fc1cf95417ff>:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  info = wave.read(path_to_file)


interactive(children=(FloatSlider(value=0.0, description='A', layout=Layout(width='800px'), max=1.0, readout_f…

<function __main__.plot_everything(A, D, S, type_A, type_D, type_R)>